In [2]:
from together import Together
import pandas as pd
import numpy as np
from datasets import load_dataset
from tokens import tokens
import os
import dspy
import textgrad
from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from textgrad.tasks import DataLoader 
import concurrent
from tqdm import tqdm
import random

c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Initialization

In [3]:
# set up tokens
os.environ['TOGETHER_API_KEY'] = together_token
os.environ['HF_TOKEN'] = hf_token

In [4]:
# Load testing data
dataset = load_dataset("gtfintechlab/fomc_communication")
sample = dataset['test'][0]
sentence = sample['sentence']
label = sample['label']
print(f"Original sentence: {sentence}\nLabel: {label}")

Original sentence: Participants agreed that the labor market had remained strong over the intermeeting period and that economic activity had risen at a moderate rate.
Label: 2


c:\Users\mikad\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Textgrad Prompt Optimization

In [5]:
def find_first_occurrence(text):
    words = ["hawkish", "dovish", "neutral"]
    positions = [(text.find(word), word) for word in words if text.find(word) != -1]
    return min(positions)[1] if positions else None

def string_based_equality_fn(prediction: Variable, ground_truth_answer: Variable):
    mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
    pred = find_first_occurrence(str(prediction.value).lower())
    if (pred != None and str(pred).lower() == str(mapping[int(ground_truth_answer.value)]).lower()):
        return 1
    else: 
        return 0

eval_fn = textgrad.autograd.StringBasedFunction(string_based_equality_fn, "Evaluate if LLM answer matches ground truth")

In [6]:
# Evaluate given sample using custom evaluation function
def eval_sample(x, y, eval_fn, model):
    x = Variable(x, requires_grad=False, role_description="query to the language model")
    y = Variable(y, requires_grad=False, role_description="correct answer for the query")
    response = model(x)
    inputs = dict(prediction = response, ground_truth_answer = y)
    eval_output_variable = eval_fn(inputs = inputs)
    return eval_output_variable
    
# Evaluate dataset using custom evaluation function
def eval_dataset(test_set, eval_fn, model, max_samples = None, max_workers = 8):
    if max_samples is None: 
        max_samples = len(test_set)
    accuracy_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, sample in enumerate(test_set):
            future = executor.submit(eval_sample, sample[0], sample[1], eval_fn, model)
            futures.append(future)
            if len(futures) >= max_samples:
                break
        tqdm_loader = tqdm(concurrent.futures.as_completed(futures), total=len(futures), position=0)
        for future in tqdm_loader:
            output = future.result()
            output = int(output.value)
            accuracy_list.append(output)
            tqdm_loader.set_description(f"Accuracy: {np.mean(accuracy_list)}")
    return accuracy_list 

# Evaluate system prompt and revert to previous prompt if new prompt has lower test accuracy
def run_test_revert(system_prompt, results, eval_fn, model, testing_dataset):
    curr_accuracy = np.mean(eval_dataset(testing_dataset, eval_fn, model))
    prev_accuracy = np.mean(results["test_accuracy"][-1])
    print(f"Test accuracy: {curr_accuracy}\nPrevious test accuracy: {prev_accuracy}")
    previous_prompt = results["prompt"][-1]

    results["new_prompt_test_accuracy"].append(curr_accuracy)
    
    if curr_accuracy < prev_accuracy:
        print(f"Rejected prompt: {system_prompt.value}")
        system_prompt.set_value(previous_prompt)
        curr_accuracy = prev_accuracy

    results["test_accuracy"].append(curr_accuracy)

In [7]:
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

starting_prompt = "Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy. Output your prediction on the first line and your explanation after."
system_prompt = Variable(starting_prompt, requires_grad=True, role_description="system prompt to the language model")
model = textgrad.BlackboxLLM(engine, system_prompt)

optimizer = TextualGradientDescent(engine=engine, parameters=[system_prompt])

In [9]:
# testing zero-shot performance
mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
data = [(data['sentence'], data['label']) for data in dataset['test']]
results = {"test_accuracy": [], "prompt": []}
results["test_accuracy"].append(eval_dataset(data, eval_fn, model))
results["prompt"].append(system_prompt.get_value())

Accuracy: 0.6088709677419355: 100%|██████████| 496/496 [00:01<00:00, 264.80it/s]


In [8]:
# subset the data to do a mini run
def pick_random(data, n = 20):
    labels = {0: [], 1: [], 2: []}
    for sentence, label in data:
        labels[label].append((sentence, label))
    result = []
    for label in labels:
        result.extend(random.sample(labels[label], n))
    return result
training_data = [(data['sentence'], data['label']) for data in dataset['train']]
testing_data = [(data['sentence'], data['label']) for data in dataset['test']]
random.shuffle(testing_data)
val_data_options = testing_data[:100]
testing_data = testing_data[100:]
training_data = pick_random(training_data, n = 30)
val_data = pick_random(val_data_options, n = 12)
testing_data = pick_random(testing_data, n = 12)

In [9]:
start_test_accuracy = np.mean(eval_dataset(testing_data, eval_fn, model))
results = {"test_accuracy": [start_test_accuracy], "prompt": [starting_prompt], 
            "new_prompt": [starting_prompt], "new_prompt_test_accuracy": [start_test_accuracy]}
print(f"Starting prompt: {starting_prompt}")
print(f"Starting test accuracy: {start_test_accuracy}")

# optimize system prompt
train_loader = DataLoader(training_data, batch_size=15, shuffle=True)

for epoch in range(3):
    for step, (batch_x, batch_y) in enumerate((pbar := tqdm(train_loader, position=0))):
        # initialize training step
        pbar.set_description(f"Epoch {epoch}: Training step {step}")
        optimizer.zero_grad()
        losses = []

        # train off items in batch
        for (x, y) in zip(batch_x, batch_y):
            eval_output = eval_sample(x, y, eval_fn, model)
            losses.append(eval_output)

        # sum losses
        total_loss = textgrad.sum(losses)
        total_loss.backward(engine)
        optimizer.step()
        
        # evaluate new system prompt
        # if worse, revert to previous prompt
        results["new_prompt"].append(system_prompt.get_value())
        print("New prompt: ", system_prompt.get_value())
        run_test_revert(system_prompt, results, eval_fn, model, testing_data)
        
        print("System prompt: ", system_prompt)
        results["prompt"].append(system_prompt.get_value())
        
        if step >= 5:
            break

Accuracy: 0.5277777777777778: 100%|██████████| 36/36 [00:00<00:00, 682.11it/s]


Starting prompt: Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy. Output your prediction on the first line and your explanation after.
Starting test accuracy: 0.5277777777777778


Epoch 0: Training step 0: : 0it [00:00, ?it/s]

New prompt:  Analyze the stance on monetary policy in the following sentence from FOMC. Determine whether it leans towards easing, tightening, or maintaining the current policy, and provide a clear explanation for your prediction, focusing on the specific words, phrases, and arguments that support your conclusion. Additionally, discuss any alternative interpretations of the sentence and explain why they are less likely.


Accuracy: 0.05555555555555555: 100%|██████████| 36/36 [00:47<00:00,  1.32s/it] 
Epoch 0: Training step 1: : 1it [03:21, 201.89s/it]

...

Test accuracy: 0.1388888888888889
Previous test accuracy: 0.7222222222222222
Rejected prompt: Analyze the given FOMC statement and determine the expected federal funds rate or the pace of balance sheet reduction, along with a clear and detailed classification of the monetary policy stance, based on the statement's context and the FOMC's overall monetary policy objectives and recent statements or actions. Explain the reasons for your prediction, focusing on the FOMC's monetary policy objectives and any indications of a shift in policy direction. Provide evidence or references that support your prediction, if possible. Discuss the potential impact on inflation, employment, and any associated risks or uncertainties, focusing on the short-term and medium-term implications. Discuss the potential risks and uncertainties associated with the predicted policy stance, and provide specific examples or scenarios that illustrate the short-term and medium-term implications of the FOMC's monetary pol

Accuracy: 0.5555555555555556: 100%|██████████| 36/36 [00:52<00:00,  1.46s/it]
Epoch 2: Training step 5: : 5it [21:48, 311.05s/it]

Test accuracy: 0.5555555555555556
Previous test accuracy: 0.7222222222222222
Rejected prompt: Analyze the given FOMC statement and determine any indications of a shift in the federal funds rate or the pace of balance sheet reduction, specifically looking for phrases like 'interest rates', 'quantitative easing', 'forward guidance', or 'open market operations'. Interpret these indications in the context of the FOMC's overall monetary policy objectives and recent statements or actions. Provide a clear and detailed classification of the expected monetary policy stance, along with a justification for your decision. Choose from the following categories: Neutral, Dovish, Moderately Hawkish, or Very Hawkish. Explain the reasons for your prediction, focusing on the FOMC's monetary policy objectives and any indications of a shift in policy direction. Provide evidence or references that support your prediction, if possible. Discuss the potential impact on inflation, employment, and any associated

Accuracy: 0.6111111111111112: 100%|██████████| 36/36 [01:06<00:00,  1.85s/it]
Epoch 2: Training step 5: : 5it [27:53, 334.66s/it]

Test accuracy: 0.6111111111111112
Previous test accuracy: 0.7222222222222222
Rejected prompt: Analyze the given FOMC statement and identify the key elements that provide insight into the FOMC's monetary policy stance on inflation, employment, and associated risks or uncertainties. Explain how these elements indicate the expected policy stance, focusing on the FOMC's monetary policy objectives and any indications of a shift in policy direction. Provide a clear and detailed classification of the monetary policy stance, along with a justification for your decision. Choose from the following categories: Neutral, Dovish, Moderately Hawkish, or Very Hawkish. Discuss the potential impact on inflation, employment, and any associated risks or uncertainties, focusing on the short-term and medium-term implications. Discuss the potential risks and uncertainties associated with the predicted policy stance, and provide specific examples or scenarios that illustrate the short-term and medium-term imp

In [11]:
pd.DataFrame(results).to_csv("textgrad.csv", index=False)

In [12]:
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

starting_prompt = "Analyze the given FOMC statement and determine the expected federal funds rate or the pace of balance sheet reduction based on the statement. Interpret the statement in the context of the FOMC's overall monetary policy objectives and recent statements or actions. Provide a clear and detailed classification of the monetary policy stance, along with a justification for your decision. Choose from the following categories: Neutral, Dovish, Moderately Hawkish, or Very Hawkish. Explain the reasons for your prediction, focusing on the FOMC's monetary policy objectives and any indications of a shift in policy direction. Provide evidence or references that support your prediction, if possible. Discuss the potential impact on inflation, employment, and any associated risks or uncertainties, focusing on the short-term and medium-term implications. Discuss the potential risks and uncertainties associated with the predicted policy stance, and provide specific examples or scenarios that illustrate the short-term and medium-term implications of the FOMC's monetary policy stance on inflation, employment, and associated risks or uncertainties."
system_prompt = Variable(starting_prompt, requires_grad=True, role_description="system prompt to the language model")
model = textgrad.BlackboxLLM(engine, system_prompt)

optimizer = TextualGradientDescent(engine=engine, parameters=[system_prompt])

mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
data = [(data['sentence'], data['label']) for data in dataset['test']]
eval_dataset(data, eval_fn, model)

Accuracy: 0.5403225806451613: 100%|██████████| 496/496 [09:35<00:00,  1.16s/it]


[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


## Testing together 1.2 API

In [4]:
# the one in prompts.py was not returning just the system prompt/user message which together 1.2 needs
def fomc_prompt(sentence: str):

    system_prompt = f"""Discard all the previous instructions. Behave like you are an expert sentence clas-
                sifier."""
    user_msg = f"""Classify the following sentence from FOMC into ‘HAWKISH’, ‘DOVISH’, or ‘NEU-
                TRAL’ class. Label ‘HAWKISH’ if it is corresponding to tightening of the monetary policy,
                ‘DOVISH’ if it is corresponding to easing of the monetary policy, or ‘NEUTRAL’ if the
                stance is neutral. Provide the label in the first line and provide a short explanation in the
                second line. This is the sentence: {sentence}"""

    return system_prompt, user_msg

In [8]:
# Testing together 1.2 API
client = Together()
model_response = client.chat.completions.create(
    model='allenai/OLMo-7B-Instruct',
    messages=[{'role': 'system', 'content': fomc_prompt(sentence)[0]},
                {'role': 'user', 'content': fomc_prompt(sentence)[1]}],
    max_tokens=128,
    temperature=0.7,
    top_k=50,
    top_p=0.7,
    repetition_penalty=1.1
)
response_label = model_response.choices[0].message.content
token_usage = {'prompt_tokens': model_response.usage.prompt_tokens, 'response_tokens': model_response.usage.completion_tokens, 'total_tokens': model_response.usage.total_tokens}
print(f"LLM response: {response_label.strip()}\nToken usage: {token_usage}")

LLM response: Neutral class
Explanation: The sentence mentions that participants agreed that the labor market remained strong and economic activity rose at a moderate rate, indicating a neutral stance towards monetary policy.
Token usage: {'prompt_tokens': 174, 'response_tokens': 37, 'total_tokens': 211}


## Testing DSPy

In [54]:
# testing dspy
# dspy together module has some issues with how they wrote it (passing specific parameters is hard)
# rate limiting error messages
# TODO: look into dspy together module more to see how to fix it

lm = dspy.Together(model = 'mistralai/Mistral-7B-Instruct-v0.3', stop = ['</s>', '<s>'])
dspy.settings.configure(lm=lm)
d = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}

class StanceAnalysis(dspy.Signature):
    """Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish."""
    
    sentence = dspy.InputField()
    stance = dspy.OutputField(desc = "hawkish, neutral, or dovish")

class Analysis(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(StanceAnalysis)
    
    def forward(self, sentence):
        return self.predict(sentence=sentence)
    
analyze = Analysis()
analyze(sentence)

Backing off 0.6 seconds after 1 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 0.5 seconds after 2 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 3.4 seconds after 3 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 7.0 seconds after 4 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 12.9 seconds after 5

KeyboardInterrupt: 

## Initial textgrad testing

In [5]:
# basic textgrad together engine
engine = get_engine('together-allenai/OLMo-7B-Instruct')
engine(fomc_prompt(sentence)[1], system_prompt = fomc_prompt(sentence)[0], max_tokens=128, temperature=0.7, top_p=0.7)

'Neutral class\nExplanation: The sentence indicates that participants agreed on the strong labor market and moderate economic activity, suggesting a neutral stance towards monetary policy.'

In [9]:
system_prompt = 'Discard all the previous instructions. Behave like you are an expert sentence classifier.'
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

sys_prompt = Variable('Discard all the previous instructions. Behave like you are an expert sentence classifier.', role_description="The system prompt")
user_prompt = Variable("Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish.", role_description="The user prompt", requires_grad=True)
input_sentence = Variable(sentence, role_description="The input sentence")
loss = TextLoss(sys_prompt, engine=engine)

# optimization does not allow you to set any parameters besides prompt & system prompt
# meaning you can't set max tokens or temperature --> problem b/c default output tokens is 2000 
# so models with too small context windows will error

optimizer = TextualGradientDescent(parameters=[user_prompt], engine=engine)
l = loss(input_sentence)
l.backward(engine)
optimizer.step()

In [10]:
# optimized version of user prompt
user_prompt.value

'Determine the monetary policy stance (hawkish, neutral, or dovish) expressed in the given sentence.'